# Parse Hyperparameters from Tune

This notebook parses hyperparameter and validation statistics from the tune runs

In [1]:
import os
import pandas as pd

!wget -O /tmp/tune.zip https://syncandshare.lrz.de/dl/fi8nRbeX8D2xP1cdu558oL6/tune.zip
!unzip -d /tmp/ -o /tmp/tune.zip > /dev/null

#!rsync -avz -e "ssh" marc@uni:/data2/breizhcrops/tune /tmp/ > /dev/null
    
tuneresults = "/tmp/tune"
runs = os.listdir(tuneresults)
runs = [run for run in runs if os.path.exists(os.path.join(tuneresults,run,"trainlog.csv"))]

--2020-04-23 10:08:17--  https://syncandshare.lrz.de/dl/fi8nRbeX8D2xP1cdu558oL6/tune.zip
Resolving syncandshare.lrz.de (syncandshare.lrz.de)... 129.187.255.213
Connecting to syncandshare.lrz.de (syncandshare.lrz.de)|129.187.255.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 855056 (835K) [application/x-zip-compressed]
Saving to: ‘/tmp/tune.zip’

/tmp/tune.zip       100%[===================>] 835,02K  1,03MB/s    in 0,8s    

2020-04-23 10:08:18 (1,03 MB/s) - ‘/tmp/tune.zip’ saved [855056/855056]



In [2]:
def parse_hyperparameters_folder_name(run):
    
    run = os.path.basename(run)
    model, *hyperparameter_str = run.split("_")

    hyperparameter = dict()
    for kv in hyperparameter_str:
        k,v = kv.split("=")
        hyperparameter[k] = v
    hyperparameter["model"] = model
    return hyperparameter

In [3]:
def read_best_epoch_stats(run):
    df = pd.read_csv(os.path.join(run,"trainlog.csv"), index_col=0)
    bestepoch = df["testloss"].idxmin()
    best_stats = df.loc[bestepoch].to_dict()
    best_stats["epoch"] = bestepoch
    return best_stats

In [4]:

def merge(dict1,dict2):
    for k,v in dict2.items():
        dict1[k] = v
    return dict1

## Parse Runs

In [5]:
results = list()
for run in runs:
    result = parse_hyperparameters_folder_name(os.path.join(tuneresults,run))
    best_epoch_stats = read_best_epoch_stats(os.path.join(tuneresults,run))
    result = merge(result,best_epoch_stats)
    results.append(result)
results = pd.DataFrame(results)

models = results.model.unique()
tables = dict()
for model in models:
    df = results.loc[results.model == model]
    tables[model] = df.sort_values(by="testloss")

## Number of Runs per Model

In [6]:
results.groupby("model").count().accuracy

model
InceptionTime         86
LSTM                  80
MSResNet              70
OmniScaleCNN          25
StarRNN               84
TempCNN               78
TransformerEncoder    39
Name: accuracy, dtype: int64

## Top 5 Runs per Model

In [16]:
tables["LSTM"].dropna(axis=1).head(5)

,accuracy,bidirectional,dropout,epoch,f1_macro,f1_micro,f1_weighted,hidden-dims,input-dim,kappa,...,precision_macro,precision_micro,precision_weighted,recall_macro,recall_micro,recall_weighted,testloss,trainloss,use-layernorm,weight-decay
339,0.768106,True,0.5713020228087161,4,0.532705,0.768106,0.763628,128,13,0.698636,...,0.521869,0.768106,0.762057,0.549575,0.768106,0.768106,0.386552,0.442643,True,5.256755602421856e-07
368,0.774519,True,0.774686260810069,3,0.534517,0.774519,0.761238,128,13,0.703580,...,0.532517,0.774519,0.759909,0.543511,0.774519,0.774519,0.390567,0.427903,True,7.450833247394745e-07
223,0.771893,True,0.5062661852286845,2,0.532838,0.771893,0.761117,64,13,0.700900,...,0.529250,0.771893,0.756474,0.539796,0.771893,0.771893,0.399834,0.504992,True,3.580983644996786e-06
409,0.768353,False,0.3736711781729697,4,0.524377,0.768353,0.752829,128,13,0.696271,...,0.516966,0.768353,0.753326,0.544729,0.768353,0.768353,0.403581,0.471659,True,5.883770066531068e-07
303,0.772638,True,0.5482919668213728,4,0.531447,0.772638,0.761888,64,13,0.702137,...,0.579313,0.772638,0.763210,0.545512,0.772638,0.772638,0.404284,0.446134,True,2.746888812711483e-08


In [17]:
tables["LSTM"].dropna(axis=1).head(5)

,accuracy,bidirectional,dropout,epoch,f1_macro,f1_micro,f1_weighted,hidden-dims,input-dim,kappa,...,precision_macro,precision_micro,precision_weighted,recall_macro,recall_micro,recall_weighted,testloss,trainloss,use-layernorm,weight-decay
339,0.768106,True,0.5713020228087161,4,0.532705,0.768106,0.763628,128,13,0.698636,...,0.521869,0.768106,0.762057,0.549575,0.768106,0.768106,0.386552,0.442643,True,5.256755602421856e-07
368,0.774519,True,0.774686260810069,3,0.534517,0.774519,0.761238,128,13,0.703580,...,0.532517,0.774519,0.759909,0.543511,0.774519,0.774519,0.390567,0.427903,True,7.450833247394745e-07
223,0.771893,True,0.5062661852286845,2,0.532838,0.771893,0.761117,64,13,0.700900,...,0.529250,0.771893,0.756474,0.539796,0.771893,0.771893,0.399834,0.504992,True,3.580983644996786e-06
409,0.768353,False,0.3736711781729697,4,0.524377,0.768353,0.752829,128,13,0.696271,...,0.516966,0.768353,0.753326,0.544729,0.768353,0.768353,0.403581,0.471659,True,5.883770066531068e-07
303,0.772638,True,0.5482919668213728,4,0.531447,0.772638,0.761888,64,13,0.702137,...,0.579313,0.772638,0.763210,0.545512,0.772638,0.772638,0.404284,0.446134,True,2.746888812711483e-08


In [18]:
tables["TransformerEncoder"].dropna(axis=1).head(5)

,accuracy,d-inner,d-model,dropout,epoch,f1_macro,f1_micro,f1_weighted,input-dim,kappa,...,num-classes,precision_macro,precision_micro,precision_weighted,recall_macro,recall_micro,recall_weighted,testloss,trainloss,weight-decay
457,0.767349,256,64,0.39907201621346594,4,0.531764,0.767349,0.761637,13,0.696568,...,9,0.554380,0.767349,0.760594,0.544132,0.767349,0.767349,0.400969,0.488115,5.523908582054716e-08
135,0.768942,512,128,0.4088349297252476,4,0.541442,0.768942,0.767045,13,0.700305,...,9,0.560923,0.768942,0.769031,0.548673,0.768942,0.768942,0.408963,0.424588,4.761209632430451e-05
375,0.759735,256,64,0.18050534550027103,3,0.535485,0.759735,0.757295,13,0.690072,...,9,0.527848,0.759735,0.762590,0.546975,0.759735,0.759735,0.418054,0.498237,2.12632196092551e-08
289,0.764531,128,32,0.2224813424255747,4,0.529045,0.764531,0.759698,13,0.693015,...,9,0.522853,0.764531,0.756520,0.538067,0.764531,0.764531,0.426122,0.481173,1.0081488940413012e-06
186,0.760768,256,64,0.4529922706712342,3,0.522875,0.760768,0.750718,13,0.686517,...,9,0.515344,0.760768,0.747352,0.534929,0.760768,0.760768,0.427807,0.526690,0.0005814957927739772


In [19]:
tables["MSResNet"].dropna(axis=1).head(5)

,accuracy,epoch,f1_macro,f1_micro,f1_weighted,hidden-dims,input-dim,kappa,learning-rate,model,num-classes,precision_macro,precision_micro,precision_weighted,recall_macro,recall_micro,recall_weighted,testloss,trainloss,weight-decay
193,0.774964,3,0.541199,0.774964,0.769642,32,13,0.706361,0.0006271686393146093,MSResNet,9,0.544985,0.774964,0.765046,0.538324,0.774964,0.774964,0.366106,0.431815,4.750234747127917e-06
314,0.778426,3,0.540204,0.778426,0.769506,128,13,0.709493,0.006578133089541308,MSResNet,9,0.539786,0.778426,0.765308,0.543615,0.778426,0.778426,0.374996,0.390789,7.220858296495318e-06
126,0.776623,3,0.535214,0.776623,0.757775,64,13,0.705113,0.0004247035855624326,MSResNet,9,0.605207,0.776623,0.767687,0.541598,0.776623,0.776623,0.380334,0.427270,3.358753712532899e-06
428,0.771033,2,0.538633,0.771033,0.769489,64,13,0.702134,0.0027681745500897606,MSResNet,9,0.605295,0.771033,0.777618,0.524294,0.771033,0.771033,0.387393,0.411190,4.034706580204553e-07
213,0.768497,3,0.545945,0.768497,0.760627,64,13,0.697944,0.008003543646192558,MSResNet,9,0.567236,0.768497,0.756437,0.539959,0.768497,0.768497,0.392848,0.447683,1.3897236942014263e-07


In [20]:
tables["TempCNN"].dropna(axis=1).head(5)

,accuracy,dropout,epoch,f1_macro,f1_micro,f1_weighted,hidden-dims,input-dim,kappa,kernelsize,...,precision_macro,precision_micro,precision_weighted,recall_macro,recall_micro,recall_weighted,sequencelenght,testloss,trainloss,weight-decay
4,0.768804,0.18203942949809093,9,0.534704,0.768804,0.765426,128,13,0.700177,7,...,0.529263,0.768804,0.763733,0.542241,0.768804,0.768804,45,0.388066,0.420825,5.181869707846283e-05
412,0.765522,0.6111322554656715,7,0.527008,0.765522,0.756577,64,13,0.693685,3,...,0.519964,0.765522,0.751619,0.536682,0.765522,0.765522,45,0.402030,0.551195,4.607371133185498e-06
371,0.768936,0.16429475058333642,4,0.531899,0.768936,0.765562,64,13,0.698956,3,...,0.530025,0.768936,0.762946,0.535743,0.768936,0.768936,45,0.403093,0.470903,2.0226716090767143e-08
185,0.769651,0.15185995136717817,4,0.532141,0.769651,0.765564,128,13,0.699058,3,...,0.565848,0.769651,0.766804,0.539146,0.769651,0.769651,45,0.403896,0.456656,3.073529220229683e-06
205,0.767457,0.0009173671298440489,1,0.523672,0.767457,0.755649,64,13,0.694051,3,...,0.532193,0.767457,0.752807,0.520714,0.767457,0.767457,45,0.405023,0.439033,1.8530056938428506e-07


In [21]:
tables["StarRNN"].dropna(axis=1).head(5)

,accuracy,epoch,f1_macro,f1_micro,f1_weighted,hidden-dims,input-dim,kappa,learning-rate,model,...,num-layers,precision_macro,precision_micro,precision_weighted,recall_macro,recall_micro,recall_weighted,testloss,trainloss,weight-decay
115,0.781731,4,0.549374,0.781731,0.776262,128,13,0.714562,0.008960989762612663,StarRNN,...,3,0.588299,0.781731,0.774753,0.547887,0.781731,0.781731,0.344780,0.396030,2.2171861339535254e-06
327,0.777861,2,0.546997,0.777861,0.773787,128,13,0.710804,0.007657585202734554,StarRNN,...,4,0.562972,0.777861,0.771784,0.549321,0.777861,0.777861,0.358919,0.443134,1.1769406596986384e-05
398,0.778149,1,0.535730,0.778149,0.768877,128,13,0.708608,0.0067575455570161504,StarRNN,...,1,0.541919,0.778149,0.765761,0.533598,0.778149,0.778149,0.370950,0.461838,5.969526064517531e-07
450,0.776851,4,0.539207,0.776851,0.772353,64,13,0.707531,0.004713412558986888,StarRNN,...,2,0.669183,0.776851,0.778383,0.526447,0.776851,0.776851,0.374779,0.366356,1.94550878954771e-06
144,0.768713,1,0.532744,0.768713,0.765329,128,13,0.698284,0.008383598081183172,StarRNN,...,1,0.526537,0.768713,0.763447,0.541427,0.768713,0.768713,0.379434,0.473511,6.694854642479949e-07


In [22]:
tables["InceptionTime"].dropna(axis=1).head(5)

,accuracy,epoch,f1_macro,f1_micro,f1_weighted,hidden-dims,input-dim,kappa,learning-rate,model,...,num-layers,precision_macro,precision_micro,precision_weighted,recall_macro,recall_micro,recall_weighted,testloss,trainloss,weight-decay
100,0.775709,4,0.542727,0.775709,0.773677,64,13,0.707494,0.0005930998594456241,InceptionTime,...,4,0.539105,0.775709,0.772233,0.547403,0.775709,0.775709,0.401794,0.438967,1.8660112778851542e-05
366,0.778191,4,0.537013,0.778191,0.759936,128,13,0.705811,0.0009980594748806762,InceptionTime,...,3,0.555022,0.778191,0.767957,0.533142,0.778191,0.778191,0.412421,0.466849,0.0010626996351948946
106,0.760011,1,0.512052,0.760011,0.750987,32,13,0.685513,0.0009386055083433407,InceptionTime,...,4,0.519447,0.760011,0.752925,0.516674,0.760011,0.760011,0.419982,0.504183,7.451748528626408e-06
353,0.759434,2,0.529544,0.759434,0.756341,32,13,0.689286,0.0018256005145633298,InceptionTime,...,4,0.520795,0.759434,0.757427,0.541913,0.759434,0.759434,0.424072,0.440606,9.727031671366588e-07
204,0.769675,4,0.530594,0.769675,0.759042,64,13,0.696469,0.00028098649833653664,InceptionTime,...,4,0.537773,0.769675,0.761159,0.530990,0.769675,0.769675,0.427361,0.430931,0.0008642986871646551


In [23]:
tables["OmniScaleCNN"].dropna(axis=1).head(5)

,accuracy,epoch,f1_macro,f1_micro,f1_weighted,kappa,learning-rate,model,precision_macro,precision_micro,precision_weighted,recall_macro,recall_micro,recall_weighted,testloss,trainloss,weight-decay
367,0.750101,3,0.516633,0.750101,0.748277,0.672344,0.001057192239267413,OmniScaleCNN,0.531053,0.750101,0.750259,0.507578,0.750101,0.750101,0.458185,0.428113,2.2522895556530792e-07
422,0.766935,2,0.527760,0.766935,0.754424,0.693273,0.003569171389539325,OmniScaleCNN,0.538308,0.766935,0.753166,0.524513,0.766935,0.766935,0.469235,0.441604,0.0023191659098454505
211,0.758959,4,0.517349,0.758959,0.737605,0.679332,0.0005707272967138424,OmniScaleCNN,0.532658,0.758959,0.755325,0.521675,0.758959,0.758959,0.469848,0.471677,1.088762753623053e-06
31,0.746603,4,0.516922,0.746603,0.746326,0.674500,0.0020118481399637603,OmniScaleCNN,0.513890,0.746603,0.759624,0.535556,0.746603,0.746603,0.477586,0.451877,2.923271245581624e-06
247,0.725087,4,0.505962,0.725087,0.720011,0.647169,0.0001188354335305461,OmniScaleCNN,0.519025,0.725087,0.742567,0.508284,0.725087,0.725087,0.484223,0.518740,4.24693550332499e-06


## Validation accuracy table

In [ ]:
metrics = ["accuracy","precision_micro","precision_macro","precision_weighted","precision_micro","recall_macro","recall_weighted", "kappa", "testloss"]
results.groupby("model").apply(lambda df: df.sort_values(by="testloss").iloc[0]).sort_values(by="testloss")[metrics]

In [31]:
metrics = ["accuracy","precision_micro","precision_macro","precision_weighted","precision_micro","recall_macro","recall_weighted", "kappa", "testloss"]
results.groupby("model").apply(lambda df: df.sort_values(by="testloss").iloc[0]).sort_values(by="testloss")[metrics]

,accuracy,precision_micro,precision_macro,precision_weighted,precision_micro,recall_macro,recall_weighted,kappa,testloss
model,,,,,,,,,
StarRNN,0.781731,0.781731,0.588299,0.774753,0.781731,0.547887,0.781731,0.714562,0.344780
MSResNet,0.774964,0.774964,0.544985,0.765046,0.774964,0.538324,0.774964,0.706361,0.366106
LSTM,0.768106,0.768106,0.521869,0.762057,0.768106,0.549575,0.768106,0.698636,0.386552
TempCNN,0.768804,0.768804,0.529263,0.763733,0.768804,0.542241,0.768804,0.700177,0.388066
TransformerEncoder,0.767349,0.767349,0.554380,0.760594,0.767349,0.544132,0.767349,0.696568,0.400969
InceptionTime,0.775709,0.775709,0.539105,0.772233,0.775709,0.547403,0.775709,0.707494,0.401794
OmniScaleCNN,0.750101,0.750101,0.531053,0.750259,0.750101,0.507578,0.750101,0.672344,0.458185
